# Set Up

In [3]:
from __future__ import print_function
from six.moves import zip, range
import pandas as pd # Handle dataframes
pd.set_option('display.max_columns', None) # Per visualizzare tutte le colonne di un dataset, display(df)
#pd.set_option('display.max_rows', None)   # Per visualizzare tutte le righe di un dataset, display(df)

import recordlinkage


import json # Lavorare con file json
from pathlib import Path # Per identificare il path del file 'config'

# Identificare il file config.json 
start = Path.cwd()
current = start.resolve()
for parent in [current] + list(current.parents):
   json_path_config_path = parent / "config.json"
   if json_path_config_path.exists():
      break 

with open(json_path_config_path) as f:
   paths = json.load(f)

import sys
sys.path.append(f'{paths["path_progetto"]}{paths["path_funzioni"]}')

from funzioni import record_linkage_title,record_linkage_city_title

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Disable future warnings


path_data_product = f'{paths["path_progetto"]}{paths["path_data_product"]}'
path_risultati = f'{paths["path_progetto"]}{paths["path_results"]}'

In [4]:
agoda = pd.read_csv(f"{path_data_product}/agoda.csv",index_col=0)
booking = pd.read_csv(f"{path_data_product}/booking.csv",index_col=0)

In [5]:
display(agoda.head(1))
display(booking.head(1))

,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
0,Raeli Hotel Lazio,Hotel Lazio Raeli,Stazione centrale Roma Termini,stazione termini,roma,0.0,159,1.0,2.0,01-08-25,02-08-25,"7,7",Ottimo,321.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08


,titolo,titolo_processed,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
0,The Sereno-3,3 Sereno The,Trionfale,trionfale,roma,3.5,242,1,2,01-08-25,02-08-25,"8,1",Ottimo,10,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,2025-01-08,2025-02-08


# Record Linkage basato esclusivamente sulla variabile TITOLO utilizzando vari thresholds

### Record linkage su titolo, threshold = 0.95

Per prima cosa un passaggio nel quale si performa ancora una piccolo cleaning sui dati.   
- Si pulisce la variabile **titolo_processed** utilizzando la funzione **clean** del pacchetto *recordlinkage*;
- Si crea una variabile **first_letter** che sarà poi utile per le strategie di **blocking**.

Inoltre si crea una copia dei dataset originali **booking** ed **agoda** per non avere problemi.

In [6]:
copia_booking = booking.copy()
copia_agoda = agoda.copy()
soglia = 0.95

# Aggiungere le colonne ID: 
copia_booking['id_booking'] = copia_booking.index
copia_agoda['id_agoda'] = copia_agoda.index

copia_booking = copia_booking.rename(columns={"titolo_processed":"titolo_booking"})
copia_agoda = copia_agoda.rename(columns={"titolo_processed":"titolo_agoda"})

# Blocking sulla prima lettera (Il blocking sulla prima lettera è una tecnica di record linkage usata per ridurre il numero di confronti tra record che devono essere effettuati.)
#  Nel record linkage, si confrontano record da due (o più) set di dati per trovare duplicati o corrispondenze. Confrontare ogni record con tutti gli altri è computazionalmente costoso, soprattutto con set grandi complessità. 
# Il blocking serve a limitare i confronti a gruppi più piccoli di record che condividono una certa caratteristica.
# Il blocking sulla prima lettera: significa che i record vengono suddivisi in blocchi in base alla prima lettera di un campo di testo, per esempio un cognome, un nome o un indirizzo. Solo i record che iniziano con la stessa lettera vengono poi confrontati tra loro.
copia_booking['first_letter'] = copia_booking['titolo_booking'].str[0]
copia_agoda['first_letter'] = copia_agoda['titolo_agoda'].str[0]

Iniziare con il record linkage.
Il record linkage viene diviso in 5 steps:
   1. Definire strategia di blocking;

   2. Generare candidate pairs secondo le regole di blocking dello step1;

   3. Configurare il metodo per il calcolo delle similarità;

   4. Calcolo delle similarità;

   5. Tramite threshold identificare match;

   6. Se una location ha più di un match si prende quella con lo score  migliore, è infatti inutile avere due     
      match diversi dato che uno sarà sicuramente sbagliato dato che la corrispondenza deve essere 1:1

   7. Aggiungere al dataframe dei match le informazioni di **agoda** e **booking**.

In [7]:
# STEP 1
# Creare un oggetto di tipo Index e  definire una strategia di "blocking" basata sul valore della colonna 'first_letter'.
indexer = recordlinkage.Index() # Creare index
indexer.block('first_letter')   # Definire il blocking sulla prima lettera (utilizzare variabile 'first_letter')

# STEP 2
# Generare i candidate pairs (coppie da confrontare) tra due dataset (copia_booking e copia_agoda), 
# secondo le regole di blocking definite prima con indexer.block() (step 1).
candidate_links = indexer.index(copia_booking, copia_agoda) # Trovare le coppie da confrontare 
print(f"Coppie candidate: {len(candidate_links)}")          # Mostrare a schermo il numero di coppie candidate

# STEP 3
# Configurare il confronto tra due colonne di testo, usando il metodo di similarità Jaro-Winkler.
# Il metodo Jaro-Winkler è particolarmente efficace per confrontare stringhe brevi e con piccole variazioni o errori di battitura (es. "Hotel Roma" vs "Hotel Roma Center").
compare = recordlinkage.Compare()
compare.string('titolo_booking',         # variabile 1 da confrontare
               'titolo_agoda',           # variabile 2 da confrontare
               method='jarowinkler',    # Metodo per il confronto
               label='name_similarity'  # Nome della variabile risultante contenente la simililarità, compresa tra 0 e 1 (0 stringhe completamente diverse, 1 stringhe identiche)
               )

# STEP 4
# Eseguire effettivamente il confronto tra le coppie di record (i candidate links) generate in precedenza (STEP 2).
# Usa le regole di similarità definite con compare
features = compare.compute(candidate_links, copia_booking, copia_agoda) # Restituisce un pandas dataframe con la similarity per ogni coppia
#print(features['name_similarity'].describe()) # Overview sulle similarities calcolate.

# STEP 5
# Trovare i match. Le coppie con un valore di similarità superiore a una soglia stabilita sono considerate match.
# Il metodo fellegi sunter non può essere usato su una sola variabile quindi si fa direttamente la selezione tramite soglia.
scores_df = features[features['name_similarity'] > soglia] # Estrazione coppie match
# Ordinare il dataset dei risultati. Aggiungere ai match le informazioni di agoda e booking
scores_df.reset_index(inplace=True)  # Fare il reset degli indici. In questo modo si avranno come variabili gli indici delle location. 
                                 # Avere gli indici come variabili faciliterà poi  le prossime operazioni.                                  
scores_df = scores_df.rename(columns={'level_0': 'index_booking', 'level_1': 'index_agoda'}) # Rinominare le variabili indice
scores_df['pair'] = scores_df['index_booking'].astype(str) + '#' + scores_df['index_agoda'].astype(str)

# Negli step successivi:
# A. Eliminati tutti gli index che hanno più di un match
# B. Selezionato solo l'index booking duplicato con score maggiore, a parità presi entrambi
# C. Selezionato solo l'index agoda duplicato con score maggiore, a parità presi entrambi
# D. Inserire nuovamente i match migliori.
# In questo modo se una location ha più di un match si prende quella con la probabilità di match migliore
# è infatti inutile avere due match diversi dato che uno sarà sicuramente sbagliato dato che la corrispondenza deve
# essere 1:1
match = scores_df.copy()


# A. Prendere solo il match migliore per ogni location, non ha senso prendere la seconda opzione.
match = scores_df.drop_duplicates(subset='index_booking', keep=False)
match = match.drop_duplicates(subset='index_agoda', keep=False)

# B.
# Filtra solo i duplicati su index_booking
dups = scores_df[scores_df.duplicated(subset='index_booking', keep=False)]
# Raggruppa per index_booking e seleziona il massimo score
booking_best_match = dups.groupby('index_booking', group_keys=False).apply(
    lambda row: row[row['name_similarity'] == row['name_similarity'].max()]
)

# C.
# Filtra solo i duplicati su index_agoda
dups = scores_df[scores_df.duplicated(subset='index_agoda', keep=False)]
# Raggruppa per index_booking e seleziona il massimo score
agoda_best_match = dups.groupby('index_agoda', group_keys=False).apply(
    lambda row: row[row['name_similarity'] == row['name_similarity'].max()]
)

# D.
agoda_best_match["pair"] = agoda_best_match['index_booking'].astype(str) + '#' + agoda_best_match['index_agoda'].astype(str)
booking_best_match["pair"] = booking_best_match['index_booking'].astype(str) + '#' + booking_best_match['index_agoda'].astype(str)
match = pd.concat([match,booking_best_match])
match = pd.concat([match,agoda_best_match])


# STEP 6
# Lista delle variabili da inserire nel dataset finale dei match. 
# Evito di inserie numero di notti e persone perchè è lo stesso per entrambi i dataset. E quindi si prende direttamente dall'ultimo dataset
variabili_comuni = ["titolo","zona","città","distanza_centro","prezzo", 
                    'recensione_voto_numerico','recensione_voto_parola','numero_recensioni'] 

# Unire il dataframe dei match con il dataframe booking. Prendere solo le informazioni delle accomodation in comune.
match = pd.merge(match, # Dataframe left
                 booking[variabili_comuni], # Dataframe right
                 left_on="index_booking",   # Il dataframe left (match) ha come indice per il merge la variabile 'index_booking'
                 right_index=True # Il dataframe right (booking) ha come indice per il merge l'index.
                 ) 

variabili_comuni.extend(['numero_notti', 'numero_persone', 'inizio_permanenza', 'fine_permanenza']) # Ora posso aggiungere le variabili comuni.
match =  pd.merge(match, # Dataframe left
                 agoda[variabili_comuni], # Dataframe right
                 left_on="index_agoda", # Il dataframe left (match) ha come indice per il merge la variabile 'index_booking'
                 right_index=True, # Il dataframe right (booking) ha come indice per il merge l'index.
                 suffixes=("_booking","_agoda") # Le variabili che avranno nome uguali in left e right avranno suffisso 'booking' in left e suffisso 'agoda' in right
                 ) 

                                                                     
# Ordinare le colonne per una visualizzazione dei dati migliore.
match = match[[ 'pair','name_similarity',
               'titolo_booking', 'titolo_agoda',
               'zona_booking','zona_agoda',
               'città_booking', 'città_agoda',
               'prezzo_booking', 'prezzo_agoda', 
               'index_booking', 'index_agoda',
               'distanza_centro_booking', 'distanza_centro_agoda',
               'recensione_voto_numerico_booking', 'recensione_voto_numerico_agoda',
               'recensione_voto_parola_booking',    'recensione_voto_parola_agoda',
               'numero_recensioni_booking','numero_recensioni_agoda',
               'numero_notti', 'numero_persone', 'inizio_permanenza', 'fine_permanenza', 
            ]]
match.drop_duplicates(inplace=True) # Rimuovere righe duplicate
match

Coppie candidate: 67915


,pair,name_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza
0,1#6,1.0,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Pantheon,Pantheon,roma,roma,322,228,1,6,0.15,0.0,"8,1","9,2",Ottimo,Eccezionale,321,4.0,1.0,2.0,01-08-25,02-08-25
1,3#391,1.0,ASTORIA GOLDEN GATE,ASTORIA GOLDEN GATE,Stazione Termini,Stazione centrale Roma Termini,roma,roma,110,110,3,391,2.00,0.0,"8,9","9,1",Favoloso,Eccezionale,1446,1512.0,1.0,2.0,01-08-25,02-08-25
2,17#288,1.0,Suite Art Navona,Suite Art Navona,Piazza Navona,Piazza Navona,roma,roma,162,162,17,288,1.00,0.0,"8,1","8,2",Ottimo,Fantastico,1176,1295.0,1.0,2.0,01-08-25,02-08-25
3,27#487,1.0,Terrace Pantheon Relais,Terrace Pantheon Relais,Pantheon,Pantheon,roma,roma,366,366,27,487,0.50,0.0,"9,3","9,2",Eccellente,Eccezionale,907,1286.0,1.0,2.0,01-08-25,02-08-25
4,29#535,1.0,Hotel Fiori,Hotel Fiori,Rione Monti,Monti,roma,roma,150,150,29,535,0.40,0.0,"8,6","8,3",Favoloso,Fantastico,718,1572.0,1.0,2.0,01-08-25,02-08-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,709#71,1.0,Hotel Genio,Genio Hotel,Piazza Navona,Piazza Navona,roma,roma,213,146,709,71,1.00,0.0,"7,7","8,3",Buono,Fantastico,530,432.0,1.0,2.0,01-08-25,02-08-25
215,800#439,1.0,Grand Hotel Palace Rome,Grand Hotel Palace Rome,Via Veneto,Via Veneto,roma,roma,332,332,800,439,1.40,0.0,"8,4","8,6",Ottimo,Fantastico,673,50.0,1.0,2.0,01-08-25,02-08-25
252,851#575,1.0,San Francesco Home,San Francesco Home,Terni,Terni,terni,terni,84,84,851,575,0.40,NaN,"8,8","9,4",Favoloso,Eccezionale,17,10.0,1.0,2.0,01-08-25,02-08-25
102,433#440,1.0,Hotel Valadier,Hotel Valadier,Spagna,Piazza di Spagna,roma,roma,249,206,433,440,1.50,0.0,"8,3","8,6",Ottimo,Fantastico,2733,3009.0,1.0,2.0,01-08-25,02-08-25


In [8]:
match.to_csv(f"{path_risultati}/matches_titolo_095.csv")

### record linkage sul titolo 0.955

Tutto il codice per la sezione precedente è stato inserito in una funzione per essere richiamato più facilemente e con soglia diversa.       
Ovviamente non si tratta di una vera e propria funzione in quanto i parametri sono molti e non tutti selezionabili, è solo un modo per risparmiare codice e rendere tutto più chiaro e semplice.

In [9]:
output_0955 = record_linkage_title( copia_agoda=agoda.copy(),     #.copy() fondamentale. Essendo i pd.DataFrame() mutable si potrebbero creare errori.
                                                                  # Infatti senza .copy() agoda subirebbe le modifiche fatte dentro la funzione
                                    copia_booking= booking.copy(),
                                    soglia = 0.955           
                                 )
display(output_0955)

,pair,name_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza
0,1#6,1.0,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Pantheon,Pantheon,roma,roma,322,228,1,6,0.15,0.0,"8,1","9,2",Ottimo,Eccezionale,321,4.0,1.0,2.0,01-08-25,02-08-25
1,3#391,1.0,ASTORIA GOLDEN GATE,ASTORIA GOLDEN GATE,Stazione Termini,Stazione centrale Roma Termini,roma,roma,110,110,3,391,2.00,0.0,"8,9","9,1",Favoloso,Eccezionale,1446,1512.0,1.0,2.0,01-08-25,02-08-25
2,17#288,1.0,Suite Art Navona,Suite Art Navona,Piazza Navona,Piazza Navona,roma,roma,162,162,17,288,1.00,0.0,"8,1","8,2",Ottimo,Fantastico,1176,1295.0,1.0,2.0,01-08-25,02-08-25
3,27#487,1.0,Terrace Pantheon Relais,Terrace Pantheon Relais,Pantheon,Pantheon,roma,roma,366,366,27,487,0.50,0.0,"9,3","9,2",Eccellente,Eccezionale,907,1286.0,1.0,2.0,01-08-25,02-08-25
4,29#535,1.0,Hotel Fiori,Hotel Fiori,Rione Monti,Monti,roma,roma,150,150,29,535,0.40,0.0,"8,6","8,3",Favoloso,Fantastico,718,1572.0,1.0,2.0,01-08-25,02-08-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,883#633,1.0,"Favolosa ""Dimora del Capriolo"" con Giardino e ...","Favolosa ""Dimora del Capriolo"" con Giardino e ...",Terni,Terni,terni,terni,216,228,883,633,2.90,NaN,"9,3","9,5",Eccellente,Eccezionale,13,13.0,1.0,2.0,01-08-25,02-08-25
271,884#605,1.0,La Severa 2,La Severa 2,Terni,Terni,terni,terni,96,96,884,605,1.40,NaN,"8,4","8,4",Ottimo,Fantastico,11,11.0,1.0,2.0,01-08-25,02-08-25
272,618#359,1.0,Hotel Patria,Hotel Patria,Stazione Termini,Stazione centrale Roma Termini,roma,roma,164,125,618,359,1.20,0.0,"8,3","8,6",Ottimo,Fantastico,1669,1841.0,1.0,2.0,01-08-25,02-08-25
273,851#575,1.0,San Francesco Home,San Francesco Home,Terni,Terni,terni,terni,84,84,851,575,0.40,NaN,"8,8","9,4",Favoloso,Eccezionale,17,10.0,1.0,2.0,01-08-25,02-08-25


### Differenze tra i due thresholds titolo

In [10]:
# Modo per individuare i match diversi
#differenza_1_new = output_city_titolo_different_threshold[~output_city_titolo_different_threshold['pair'].isin(match['pair'])]
match_diversi = list(set(match.pair).difference(set(output_0955.pair)))
print(f"Numero di match diversi: {len(match_diversi)}")


match_diversi_df = match[match["pair"].isin(match_diversi)]
print(match_diversi_df.shape)
display(match_diversi_df)

Numero di match diversi: 0
(0, 24)


,pair,name_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza


Non c'è differenza tra i due threshold.

# Record linkage su titolo + città

### Record linkage città - titolo. Threshold titolo a 0.95

Iniziare con il record linkage, dopo una breve fase di preprocessing
Il record linkage viene diviso in 7 steps:
   1. Definire strategia di blocking;

   2. Generare candidate pairs secondo le regole di blocking dello step1;

   3. Configurare il metodo per il calcolo delle similarità;

   4. Calcolo delle similarità;

   5. Preparare dati  (binarizzazione) per l'ECMClassifier (Fellegi-Sunter);

   6. Tramite ECMClassifier (Fellegi-Sunter) identificare match;

   7. Se una location ha più di un match si prende quella con lo score  migliore, è infatti inutile avere due     
      match diversi dato che uno sarà sicuramente sbagliato dato che la corrispondenza deve essere 1:1


   8. Aggiungere al dataframe dei match le informazioni di **agoda** e **booking**.

In [11]:
copia_booking = booking.copy()
copia_agoda = agoda.copy()

# Aggiungere le colonne ID: 
copia_booking['id_booking'] = copia_booking.index
copia_agoda['id_agoda'] = copia_agoda.index

copia_booking = copia_booking.rename(columns={"titolo_processed":"titolo_booking"})
copia_agoda = copia_agoda.rename(columns={"titolo_processed":"titolo_agoda"})

# Blocking sulla prima lettera (Il blocking sulla prima lettera è una tecnica di record linkage usata per ridurre il numero di confronti tra record che devono essere effettuati.)
#  Nel record linkage, si confrontano record da due (o più) set di dati per trovare duplicati o corrispondenze. Confrontare ogni record con tutti gli altri è computazionalmente costoso, soprattutto con set grandi complessità. 
# Il blocking serve a limitare i confronti a gruppi più piccoli di record che condividono una certa caratteristica.
# Il blocking sulla prima lettera: significa che i record vengono suddivisi in blocchi in base alla prima lettera di un campo di testo, per esempio un cognome, un nome o un indirizzo. Solo i record che iniziano con la stessa lettera vengono poi confrontati tra loro.
copia_booking['first_letter'] = copia_booking['titolo_booking'].str[0]
copia_agoda['first_letter'] = copia_agoda['titolo_agoda'].str[0]
copia_agoda.head(1)

,titolo,titolo_agoda,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime,id_agoda,first_letter
0,Raeli Hotel Lazio,Hotel Lazio Raeli,Stazione centrale Roma Termini,stazione termini,roma,0.0,159,1.0,2.0,01-08-25,02-08-25,"7,7",Ottimo,321.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08,0,H


In [12]:
# STEP 0, impostare i threshold/soglie
soglia_titolo = 0.95
soglia_città = 0.8

# STEP 1, strategia di blocking
indexer = recordlinkage.Index()
indexer.block('first_letter')


# STEP 2, candidate pairs
candidate_links = indexer.index(copia_booking, copia_agoda)


# STEP 3, configurare metodo per il calcolo delle similarità
compare = recordlinkage.Compare()
compare.string('titolo_booking', 'titolo_agoda', method='jarowinkler', label='name_similarity')
compare.string('città', 'città', method='jarowinkler', label='città_similarity')

# STEP 4, calcolo similarità
features = compare.compute(candidate_links, copia_booking, copia_agoda)


# STEP 5 
# Binarizzazione per Fellegi-Sunter (lascia più margine)
# Il modello si basa su variabili binarie che indicano se un certo campo matcha (coincide) o meno — o almeno supera una certa soglia di somiglianza.
# Il modello Fellegi-Sunter lavora meglio (o richiede) variabili binarie:
# Invece di usare direttamente la similarità continua (es. 0.96, 0.87...), si trasforma in vero/falso in base a una soglia ritenuta significativa.
# Questo semplifica il calcolo delle probabilità di match e non-match, che nel modello sono basate su matrici di confusione binarie (es. probabilità che name_similarity=True dato che è un match vs. non match).
features_bin = features.copy()
features_bin["name_similarity"] = features_bin["name_similarity"] > soglia_titolo
features_bin["città_similarity"] = features_bin["città_similarity"] > soglia_città

# STEP 6 Classificare tramite  ECMClassifier (Fellegi-Sunter) e trovare match.
fs = recordlinkage.ECMClassifier() # Definire il classificatore
fs.fit(features_bin)               # Train classificatore
matches = fs.predict(features_bin) # Ottenere i match

# calcola punteggio medio come proxy (media delle similarità)
scores = features.loc[list(matches)].mean(axis=1)

# Creare un dataframe con gli scores e rinominare le variabili
scores_df = scores.reset_index()
scores_df.columns = ['index_booking', 'index_agoda', 'score'] # Siccome l'ordine di confronto era booking e poi agoda, il primo indice è quello di booking, segue l'index di agoda e lo score


# Negli step successivi:
# A. Eliminati tutti gli index che hanno più di un match
# B. Selezionato solo l'index booking duplicato con score maggiore, a parità presi entrambi
# C. Selezionato solo l'index agoda duplicato con score maggiore, a parità presi entrambi
# D. Inserire nuovamente i match migliori.
# In questo modo se una location ha più di un match si prende quella con la probabilità di match migliore
# è infatti inutile avere due match diversi dato che uno sarà sicuramente sbagliato dato che la corrispondenza deve
# essere 1:1
match = scores_df.copy()


# A. Prendere solo il match migliore per ogni location, non ha senso prendere la seconda opzione.
match = scores_df.drop_duplicates(subset='index_booking', keep=False)
match = match.drop_duplicates(subset='index_agoda', keep=False)

# B.
# Filtra solo i duplicati su index_booking
dups = scores_df[scores_df.duplicated(subset='index_booking', keep=False)]

# Raggruppa per index_booking e seleziona il massimo score
booking_best_match = dups.groupby('index_booking', group_keys=False).apply(
    lambda row: row[row['score'] == row['score'].max()]
)

# C.
# Filtra solo i duplicati su index_agoda
dups = scores_df[scores_df.duplicated(subset='index_agoda', keep=False)]
# Raggruppa per index_booking e seleziona il massimo score
agoda_best_match = dups.groupby('index_agoda', group_keys=False).apply(
    lambda row: row[row['score'] == row['score'].max()]
)

# D.
agoda_best_match["pair"] = agoda_best_match['index_booking'].astype(str) + '#' + agoda_best_match['index_agoda'].astype(str)
booking_best_match["pair"] = booking_best_match['index_booking'].astype(str) + '#' + booking_best_match['index_agoda'].astype(str)
match = pd.concat([match,booking_best_match])
match = pd.concat([match,agoda_best_match])



# STEP 7, aggiungere informazioni utili ai risultati
match['pair'] = match['index_booking'].astype(str) + '#' + match['index_agoda'].astype(str)

# Lista delle variabili da inserire nel dataset finale dei match. 
# Evito di inserie numero di notti e persone perchè è lo stesso per entrambi i dataset. E quindi si prende direttamente dall'ultimo dataset
variabili_comuni = ["titolo","zona","città","distanza_centro","prezzo", 
                    'recensione_voto_numerico','recensione_voto_parola','numero_recensioni'] 

# Unire il dataframe dei match con il dataframe booking. Prendere solo le informazioni delle accomodation in comune.
match = pd.merge(match, # Dataframe left
                 booking[variabili_comuni], # Dataframe right
                 left_on="index_booking",   # Il dataframe left (match) ha come indice per il merge la variabile 'index_booking'
                 right_index=True # Il dataframe right (booking) ha come indice per il merge l'index.
                 ) 

variabili_comuni.extend(['numero_notti', 'numero_persone', 'inizio_permanenza', 'fine_permanenza']) # Ora posso aggiungere le variabili comuni.
match =  pd.merge(match, # Dataframe left
                 agoda[variabili_comuni], # Dataframe right
                 left_on="index_agoda", # Il dataframe left (match) ha come indice per il merge la variabile 'index_booking'
                 right_index=True, # Il dataframe right (booking) ha come indice per il merge l'index.
                 suffixes=("_booking","_agoda") # Le variabili che avranno nome uguali in left e right avranno suffisso 'booking' in left e suffisso 'agoda' in right
                 ) 

# Estrarre gli score della città e del titolo per poi aggiungerli al dataset finale
single_scores = features.reset_index()
single_scores["pair"] = single_scores["level_0"].astype(str) + '#' + single_scores["level_1"].astype(str)
match = pd.merge(match,single_scores,on="pair")

                                                                     
# Ordinare le colonne per una visualizzazione dei dati migliore.
match = match[[ 'pair','score','name_similarity','città_similarity',
               'titolo_booking', 'titolo_agoda',
               'zona_booking','zona_agoda',
               'città_booking', 'città_agoda',
               'prezzo_booking', 'prezzo_agoda', 
               'index_booking', 'index_agoda',
               'distanza_centro_booking', 'distanza_centro_agoda',
               'recensione_voto_numerico_booking', 'recensione_voto_numerico_agoda',
               'recensione_voto_parola_booking',    'recensione_voto_parola_agoda',
               'numero_recensioni_booking','numero_recensioni_agoda',
               'numero_notti', 'numero_persone', 'inizio_permanenza', 'fine_permanenza', 
            ]]

match.drop_duplicates(inplace=True) # Rimuovere righe duplicate totalmente
display(match.head(2))
print(match.shape)

,pair,score,name_similarity,città_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza
0,1#6,1.0,1.0,1.0,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Pantheon,Pantheon,roma,roma,322,228,1,6,0.15,0.0,"8,1","9,2",Ottimo,Eccezionale,321,4.0,1.0,2.0,01-08-25,02-08-25
1,3#391,1.0,1.0,1.0,ASTORIA GOLDEN GATE,ASTORIA GOLDEN GATE,Stazione Termini,Stazione centrale Roma Termini,roma,roma,110,110,3,391,2.00,0.0,"8,9","9,1",Favoloso,Eccezionale,1446,1512.0,1.0,2.0,01-08-25,02-08-25


(275, 26)


In [13]:
# Esportare i risultati
match.to_csv(f'{path_risultati}/matches_titolo_095_città_08.csv', index=False)

### Record linkage città - titolo. Threshold titolo a 0.95

In [14]:
output_city_titolo_different_threshold = record_linkage_city_title( dataframe_booking = booking.copy(), #.copy() fondamentale. 
                                                   # Essendo i pd.DataFrame() mutable si potrebbero creare errori.
                                                   # Infatti senza .copy() agoda subirebbe le modifiche fatte dentro la funzione
                                                   dataframe_agoda = agoda.copy(),
                                                   soglia_città = 0.9,
                                                   soglia_titolo = 0.95      
                                                )
display(output_city_titolo_different_threshold.head(2))
print(output_city_titolo_different_threshold.shape)

,pair,score,name_similarity,città_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza
0,1#6,1.0,1.0,1.0,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Sonder by Marriott Bonvoy Piazza Venezia Apart...,Pantheon,Pantheon,roma,roma,322,228,1,6,0.15,0.0,"8,1","9,2",Ottimo,Eccezionale,321,4.0,1.0,2.0,01-08-25,02-08-25
1,3#391,1.0,1.0,1.0,ASTORIA GOLDEN GATE,ASTORIA GOLDEN GATE,Stazione Termini,Stazione centrale Roma Termini,roma,roma,110,110,3,391,2.00,0.0,"8,9","9,1",Favoloso,Eccezionale,1446,1512.0,1.0,2.0,01-08-25,02-08-25


(275, 26)


### Differenze tra i due thresholds della città

Il threshold del titolo è rimasto lo stesso, è cambiato quello della città.
Con un threshold di città più alto ci sono 9 match in più, il che è strano.
Si controllino le coppie matchate diverse tra i due dataset.

In [15]:
# Un altro modo per individuare i match diversi
#differenza_1_new = output_city_titolo_different_threshold[~output_city_titolo_different_threshold['pair'].isin(match['pair'])]

match_diversi = list(set(output_city_titolo_different_threshold.pair).difference(set(match.pair)))
match_diversi

[]

In [16]:
output_city_titolo_different_threshold[output_city_titolo_different_threshold["pair"].isin(match_diversi)]

,pair,score,name_similarity,città_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza


Modificando il threshold della città il risultato non cambia, questo accade perchè le due città prese in considerazione hanno nomi profondamente diversi, ovvero Roma e Terni.   
Con città aventi nomi più simili i risultati sarebbero potuti cambiare con threshold diversi, più o meno rigidi.

### Differenze di match tra link analysis con la sola variabile titolo e link analysis con due variabili, città e titolo

In [17]:
matches_titolo_095 = pd.read_csv(f"{path_risultati}/matches_titolo_095.csv")
matches_titolo_095_città_08 = pd.read_csv(f"{path_risultati}/matches_titolo_095_città_08.csv")

In [18]:
print(f"Numero di match con solo titolo: {matches_titolo_095.shape}")
print(f"Numero di match con titolo e città: {matches_titolo_095_città_08.shape}")

Numero di match con solo titolo: (275, 25)
Numero di match con titolo e città: (275, 26)


In [19]:
# Modo per individuare i match diversi
#differenza_1_new = output_city_titolo_different_threshold[~output_city_titolo_different_threshold['pair'].isin(match['pair'])]
match_diversi = list(set(matches_titolo_095.pair).difference(set(matches_titolo_095_città_08.pair)))
print(f"Numero di match diversi: {len(match_diversi)}")


match_diversi_df = matches_titolo_095[matches_titolo_095["pair"].isin(match_diversi)]
print(match_diversi_df.shape)
display(match_diversi_df)

Numero di match diversi: 0
(0, 25)


,Unnamed: 0,pair,name_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza


Non ci sono match diversi.

# Record linkage su titolo + città + zona

Iniziare con il record linkage, dopo una breve fase di preprocessing
Il record linkage viene diviso in 7 steps:
   1. Definire strategia di blocking;

   2. Generare candidate pairs secondo le regole di blocking dello step1;

   3. Configurare il metodo per il calcolo delle similarità;

   4. Calcolo delle similarità;

   5. Preparare dati  (binarizzazione) per l'ECMClassifier (Fellegi-Sunter);

   6. Tramite ECMClassifier (Fellegi-Sunter) identificare match;

   7. Se una location ha più di un match si prende quella con lo score  migliore, è infatti inutile avere due     
      match diversi dato che uno sarà sicuramente sbagliato dato che la corrispondenza deve essere 1:1


   8. Aggiungere al dataframe dei match le informazioni di **agoda** e **booking**.


Analisi estesa a 3 variabili:
   - titolo

   - città

   - zona

In [20]:
copia_booking = booking.copy()
copia_agoda = agoda.copy()

# Aggiungere le colonne ID: 
copia_booking['id_booking'] = copia_booking.index
copia_agoda['id_agoda'] = copia_agoda.index

copia_booking = copia_booking.rename(columns={"titolo_processed":"titolo_booking"})
copia_agoda = copia_agoda.rename(columns={"titolo_processed":"titolo_agoda"})

# Blocking sulla prima lettera (Il blocking sulla prima lettera è una tecnica di record linkage usata per ridurre il numero di confronti tra record che devono essere effettuati.)
#  Nel record linkage, si confrontano record da due (o più) set di dati per trovare duplicati o corrispondenze. Confrontare ogni record con tutti gli altri è computazionalmente costoso, soprattutto con set grandi complessità. 
# Il blocking serve a limitare i confronti a gruppi più piccoli di record che condividono una certa caratteristica.
# Il blocking sulla prima lettera: significa che i record vengono suddivisi in blocchi in base alla prima lettera di un campo di testo, per esempio un cognome, un nome o un indirizzo. Solo i record che iniziano con la stessa lettera vengono poi confrontati tra loro.
copia_booking['first_letter'] = copia_booking['titolo_booking'].str[0]
copia_agoda['first_letter'] = copia_agoda['titolo_agoda'].str[0]
copia_agoda.head(1)

,titolo,titolo_agoda,zona,zona_processed,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime,id_agoda,first_letter
0,Raeli Hotel Lazio,Hotel Lazio Raeli,Stazione centrale Roma Termini,stazione termini,roma,0.0,159,1.0,2.0,01-08-25,02-08-25,"7,7",Ottimo,321.0,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08,0,H


In [21]:
# STEP 0, impostare i threshold/soglie
soglia_titolo = 0.95
soglia_città = 0.8
soglia_zona = 0.95

# STEP 1, strategia di blocking
indexer = recordlinkage.Index()
indexer.block('first_letter')


# STEP 2, candidate pairs
candidate_links = indexer.index(copia_booking, copia_agoda)


# STEP 3, configurare metodo per il calcolo delle similarità
compare = recordlinkage.Compare()
compare.string('titolo_booking', 'titolo_agoda', method='jarowinkler', label='name_similarity')
compare.string('città', 'città', method='jarowinkler', label='città_similarity')
compare.string('zona_processed', 'zona_processed', method='jarowinkler', label='zona_similarity')

# STEP 4, calcolo similarità
features = compare.compute(candidate_links, copia_booking, copia_agoda)


# STEP 5 
# Binarizzazione per Fellegi-Sunter (lascia più margine)
# Il modello si basa su variabili binarie che indicano se un certo campo matcha (coincide) o meno — o almeno supera una certa soglia di somiglianza.
# Il modello Fellegi-Sunter lavora meglio (o richiede) variabili binarie:
# Invece di usare direttamente la similarità continua (es. 0.96, 0.87...), si trasforma in vero/falso in base a una soglia ritenuta significativa.
# Questo semplifica il calcolo delle probabilità di match e non-match, che nel modello sono basate su matrici di confusione binarie (es. probabilità che name_similarity=True dato che è un match vs. non match).
features_bin = features.copy()
features_bin["name_similarity"] = features_bin["name_similarity"] > soglia_titolo
features_bin["città_similarity"] = features_bin["città_similarity"] > soglia_città
features_bin["zona_similarity"] = features_bin["zona_similarity"] > soglia_zona


# STEP 6 Classificare tramite  ECMClassifier (Fellegi-Sunter) e trovare match.
fs = recordlinkage.ECMClassifier() # Definire il classificatore
fs.fit(features_bin)               # Train classificatore
matches = fs.predict(features_bin) # Ottenere i match

# calcola punteggio medio come proxy (media delle similarità)
scores = features.loc[list(matches)].mean(axis=1)

# Creare un dataframe con gli scores e rinominare le variabili
scores_df = scores.reset_index()
scores_df.columns = ['index_booking', 'index_agoda', 'score'] # Siccome l'ordine di confronto era booking e poi agoda, il primo indice è quello di booking, segue l'index di agoda e lo score


# Negli step successivi:
# A. Eliminati tutti gli index che hanno più di un match
# B. Selezionato solo l'index booking duplicato con score maggiore, a parità presi entrambi
# C. Selezionato solo l'index agoda duplicato con score maggiore, a parità presi entrambi
# D. Inserire nuovamente i match migliori.
# In questo modo se una location ha più di un match si prende quella con la probabilità di match migliore
# è infatti inutile avere due match diversi dato che uno sarà sicuramente sbagliato dato che la corrispondenza deve
# essere 1:1
match = scores_df.copy()


# A. Prendere solo il match migliore per ogni location, non ha senso prendere la seconda opzione.
match = scores_df.drop_duplicates(subset='index_booking', keep=False)
match = match.drop_duplicates(subset='index_agoda', keep=False)

# B.
# Filtra solo i duplicati su index_booking
dups = scores_df[scores_df.duplicated(subset='index_booking', keep=False)]

# Raggruppa per index_booking e seleziona il massimo score
booking_best_match = dups.groupby('index_booking', group_keys=False).apply(
    lambda row: row[row['score'] == row['score'].max()]
)

# C.
# Filtra solo i duplicati su index_agoda
dups = scores_df[scores_df.duplicated(subset='index_agoda', keep=False)]
# Raggruppa per index_booking e seleziona il massimo score
agoda_best_match = dups.groupby('index_agoda', group_keys=False).apply(
    lambda row: row[row['score'] == row['score'].max()]
)

# D.
agoda_best_match["pair"] = agoda_best_match['index_booking'].astype(str) + '#' + agoda_best_match['index_agoda'].astype(str)
booking_best_match["pair"] = booking_best_match['index_booking'].astype(str) + '#' + booking_best_match['index_agoda'].astype(str)
match = pd.concat([match,booking_best_match])
match = pd.concat([match,agoda_best_match])



# STEP 7, aggiungere informazioni utili ai risultati
match['pair'] = match['index_booking'].astype(str) + '#' + match['index_agoda'].astype(str)

# Lista delle variabili da inserire nel dataset finale dei match. 
# Evito di inserie numero di notti e persone perchè è lo stesso per entrambi i dataset. E quindi si prende direttamente dall'ultimo dataset
variabili_comuni = ["titolo","zona","città","distanza_centro","prezzo", 
                    'recensione_voto_numerico','recensione_voto_parola','numero_recensioni'] 

# Unire il dataframe dei match con il dataframe booking. Prendere solo le informazioni delle accomodation in comune.
match = pd.merge(match, # Dataframe left
                 booking[variabili_comuni], # Dataframe right
                 left_on="index_booking",   # Il dataframe left (match) ha come indice per il merge la variabile 'index_booking'
                 right_index=True # Il dataframe right (booking) ha come indice per il merge l'index.
                 ) 

variabili_comuni.extend(['numero_notti', 'numero_persone', 'inizio_permanenza', 'fine_permanenza']) # Ora posso aggiungere le variabili comuni.
match =  pd.merge(match, # Dataframe left
                 agoda[variabili_comuni], # Dataframe right
                 left_on="index_agoda", # Il dataframe left (match) ha come indice per il merge la variabile 'index_booking'
                 right_index=True, # Il dataframe right (booking) ha come indice per il merge l'index.
                 suffixes=("_booking","_agoda") # Le variabili che avranno nome uguali in left e right avranno suffisso 'booking' in left e suffisso 'agoda' in right
                 ) 

# Estrarre gli score della città e del titolo per poi aggiungerli al dataset finale
single_scores = features.reset_index()
single_scores["pair"] = single_scores["level_0"].astype(str) + '#' + single_scores["level_1"].astype(str)
match = pd.merge(match,single_scores,on="pair")

                                                                     
# Ordinare le colonne per una visualizzazione dei dati migliore.
match = match[[ 'pair','score','name_similarity','città_similarity','zona_similarity',
               'titolo_booking', 'titolo_agoda',
               'zona_booking','zona_agoda',
               'città_booking', 'città_agoda',
               'prezzo_booking', 'prezzo_agoda', 
               'index_booking', 'index_agoda',
               'distanza_centro_booking', 'distanza_centro_agoda',
               'recensione_voto_numerico_booking', 'recensione_voto_numerico_agoda',
               'recensione_voto_parola_booking',    'recensione_voto_parola_agoda',
               'numero_recensioni_booking','numero_recensioni_agoda',
               'numero_notti', 'numero_persone', 'inizio_permanenza', 'fine_permanenza', 
            ]]

match.drop_duplicates(inplace=True) # Rimuovere righe duplicate totalmente
display(match.head())
print(match.shape)

,pair,score,name_similarity,città_similarity,zona_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza
0,16#480,0.960670,0.882011,1.0,1.000000,MC Guest House,The Right Place - Guest House,San Giovanni,San Giovanni,roma,roma,84,129,16,480,2.5,0.7,"8,1","8,9",Ottimo,Fantastico,538,22.0,1.0,2.0,01-08-25,02-08-25
1,25#126,0.923648,0.770945,1.0,1.000000,Oscar Rooms,Suite Opera Rooms ...,Stazione Termini,Stazione centrale Roma Termini,roma,roma,94,117,25,126,1.6,0.0,"7,3","7,9",Buono,Ottimo,788,94.0,1.0,2.0,01-08-25,02-08-25
2,27#487,1.000000,1.000000,1.0,1.000000,Terrace Pantheon Relais,Terrace Pantheon Relais,Pantheon,Pantheon,roma,roma,366,366,27,487,0.5,0.0,"9,3","9,2",Eccellente,Eccezionale,907,1286.0,1.0,2.0,01-08-25,02-08-25
3,44#142,0.971429,1.000000,1.0,0.914286,Hotel Relais Dei Papi,Hotel Relais Dei Papi,Vaticano Prati,Città del Vaticano,roma,roma,264,226,44,142,2.1,0.0,"8,2","8,8",Ottimo,Fantastico,3342,34.0,1.0,2.0,01-08-25,02-08-25
4,69#516,0.806217,1.000000,1.0,0.418651,Hotel Abitart,Abitart Hotel,Aventino,Eur e Garbatella,roma,roma,205,156,69,516,2.6,1.4,"8,5","8,6",Ottimo,Fantastico,2515,2640.0,1.0,2.0,01-08-25,02-08-25


(740, 27)


In [22]:
match.tail()

,pair,score,name_similarity,città_similarity,zona_similarity,titolo_booking,titolo_agoda,zona_booking,zona_agoda,città_booking,città_agoda,prezzo_booking,prezzo_agoda,index_booking,index_agoda,distanza_centro_booking,distanza_centro_agoda,recensione_voto_numerico_booking,recensione_voto_numerico_agoda,recensione_voto_parola_booking,recensione_voto_parola_agoda,numero_recensioni_booking,numero_recensioni_agoda,numero_notti,numero_persone,inizio_permanenza,fine_permanenza
959,817#659,0.891176,0.673529,1.0,1.0,Cospea B&B,La Casa nel Borgo,Terni,Terni,terni,terni,55,120,817,659,1.7,NaN,"9,0","9,6",Eccellente,Eccezionale,674,35.0,1.0,2.0,01-08-25,02-08-25
961,835#662,0.878082,0.634247,1.0,1.0,Le Camere di Ettore,Marmore Charming House Greenway,Terni,Terni,terni,terni,79,120,835,662,1.1,NaN,"8,9","9,3",Favoloso,Eccezionale,103,83.0,1.0,2.0,01-08-25,02-08-25
963,852#666,0.885802,0.657407,1.0,1.0,Tre Monumenti Luxury Suites,Luce Marmore,Terni,Terni,terni,terni,113,162,852,666,1.0,NaN,10,"9,1",Eccezionale,Eccezionale,3,45.0,1.0,2.0,01-08-25,02-08-25
966,876#670,0.948690,0.846070,1.0,1.0,Casa Vacanze Chiara,Casa Vacanze Piediluco,Terni,Terni,terni,terni,104,125,876,670,3.5,NaN,"9,7","9,1",Eccezionale,Eccezionale,3,0.0,1.0,2.0,01-08-25,02-08-25
967,858#673,0.883912,0.651736,1.0,1.0,"Interamna House, intero appartamento exclusive...",Salute Apartments,Terni,Terni,terni,terni,97,103,858,673,0.5,NaN,"9,0","8,9",Eccellente,Fantastico,3,123.0,1.0,2.0,01-08-25,02-08-25


è chiaro che con l'aggiunta della zona vengono matchate molte location in modo errato.    
Ad esempio: 
   - Oscar Rooms e Suite Opera Rooms (ROMA)
   - Tre Monumenti Luxury Suites e	Luce Marmore (TERNI)


Inoltre per quanto riguarda i dati relativi alla città di Terni la zona non è una variabile  rappresentativa  (quasi sempre uguale alla variabile città) perciò aggiungerla falsa l'analisi.

In conclusione l'aggiunta della variabile **zona** non è utile.